In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
style.use('fivethirtyeight')
import matplotlib.cm as cm
import helper

In [2]:
colors = ["#FF0B04", "#F1BE48",
           "#B9975B", "#8B5B29",
           "#524727",
         ]
sns.set_palette(sns.color_palette(colors))

In [3]:
df = pd.read_csv('Ames_Housing_Price_Data.csv', 
                             index_col=0,low_memory = False)

In [4]:
train, test = helper.data_processing_wrapper(df, num_to_cat_list=[], remove_PID=False)

In [5]:
train['LogSalePrice'] = np.log(train['SalePrice'])
test['LogSalePrice'] = np.log(test['SalePrice'])

In [6]:
nhds = train.groupby('Neighborhood').median()[['LogSalePrice', 'GrLivArea']]

In [7]:
weights = train.groupby('Neighborhood').count().apply(lambda x: x['PID']/len(train) ,axis=1).to_list()

In [8]:
scaler = StandardScaler()
_ = scaler.fit_transform(nhds)
clusterer = KMeans(n_clusters=2, random_state=42)
cluster_labels = clusterer.fit_predict(_, sample_weight=weights)
nhds['Cluster'] = cluster_labels

In [9]:
cluster_dict = nhds['Cluster'].to_dict()

In [10]:
cluster_dict

{'Blmngtn': 0,
 'Blueste': 1,
 'BrDale': 1,
 'BrkSide': 1,
 'ClearCr': 0,
 'CollgCr': 0,
 'Crawfor': 0,
 'Edwards': 1,
 'Gilbert': 0,
 'Greens': 1,
 'GrnHill': 0,
 'IDOTRR': 1,
 'Landmrk': 1,
 'MeadowV': 1,
 'Mitchel': 1,
 'NAmes': 1,
 'NPkVill': 1,
 'NWAmes': 0,
 'NoRidge': 0,
 'NridgHt': 0,
 'OldTown': 1,
 'SWISU': 1,
 'Sawyer': 1,
 'SawyerW': 0,
 'Somerst': 0,
 'StoneBr': 0,
 'Timber': 0,
 'Veenker': 0}

In [16]:
train['NhdCluster'] = train.apply(lambda x: cluster_dict[x['Neighborhood']], axis=1)
test['NhdCluster'] = test.apply(lambda x: cluster_dict[x['Neighborhood']], axis=1)

In [17]:
comp_dict = train.groupby(['Neighborhood', 'BedroomAbvGr', 'BldgType',
               'OverallQual', 'FullBath', 'KitchenQual', 'GarageCars']).mean()['LogSalePrice'].to_dict()

In [18]:
train['Comp'] = train.apply(lambda x: comp_dict[(x['Neighborhood'], x['BedroomAbvGr'], x['BldgType'],
               x['OverallQual'], x['FullBath'], x['KitchenQual'], x['GarageCars'])], axis=1)

In [19]:
alt_dict = train.groupby('Neighborhood').mean()['LogSalePrice'].to_dict()

In [20]:
def test_comp(x):
    if (x['Neighborhood'], x['BedroomAbvGr'], x['BldgType'],
               x['OverallQual'], x['FullBath'], x['KitchenQual'], x['GarageCars']) in comp_dict.keys():
        return comp_dict[(x['Neighborhood'], x['BedroomAbvGr'], x['BldgType'],
               x['OverallQual'], x['FullBath'], x['KitchenQual'], x['GarageCars'])]
    else:
        return alt_dict[x['Neighborhood']]    

In [21]:
test['Comp'] = test.apply(lambda x: test_comp(x), axis=1)

In [22]:
X0_train = train.loc[train['NhdCluster']==0,:].drop(['SalePrice', 'LogSalePrice', 'PID', 'TotalBsmtSF', 'GrLivArea', 'NhdCluster'], axis=1)
y0_train = train.loc[train['NhdCluster']==0, 'LogSalePrice']
X0_test = test.loc[test['NhdCluster']==0,:].drop(['SalePrice', 'LogSalePrice', 'PID', 'TotalBsmtSF', 'GrLivArea', 'NhdCluster'], axis=1)
y0_test = test.loc[test['NhdCluster']==0, 'LogSalePrice']
X1_train = train.loc[train['NhdCluster']==1,:].drop(['SalePrice', 'LogSalePrice', 'PID', 'TotalBsmtSF', 'GrLivArea', 'NhdCluster'], axis=1)
y1_train = train.loc[train['NhdCluster']==1, 'LogSalePrice']
X1_test = test.loc[test['NhdCluster']==1,:].drop(['SalePrice', 'LogSalePrice', 'PID', 'TotalBsmtSF', 'GrLivArea', 'NhdCluster'], axis=1)
y1_test = test.loc[test['NhdCluster']==1, 'LogSalePrice']

In [23]:
print(X0_train.shape)
print(X1_train.shape)
print('\n')
print(y0_train.shape)
print(y1_train.shape)
print('\n')
print(X0_test.shape)
print(X1_test.shape)
print('\n')
print(y0_test.shape)
print(y1_test.shape)

(896, 78)
(975, 78)


(896,)
(975,)


(298, 78)
(326, 78)


(298,)
(326,)


In [25]:
categorical = [col for col in train.select_dtypes(['object','bool']).columns.to_list()]

## Lasso for selection.

In [70]:
def Lasso_select(X, y, alpha):

    pipe = Pipeline(
        [
            ('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown = 'ignore'), categorical)], 
                                                remainder='passthrough')),
            ('scaler', StandardScaler(with_mean=False))
        ]
    )
    
    X = pipe.fit_transform(X)

    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    cross = cross_val_score(Lasso(alpha=alpha, max_iter=5000), X, y, scoring='r2', cv=cv, n_jobs=-1)
    
    selector = SelectFromModel(Lasso(alpha=alpha, max_iter=5000))
    selector.fit(X,y)
    num_features = np.sum(selector.get_support())
    
    return cross, num_features

In [71]:
for alpha in np.logspace(-5, -1, 5):
    print(Lasso_select(X0_train,y0_train,alpha))

(array([0.93582159, 0.92170681, 0.90819949, 0.91748462, 0.92919149]), 176)
(array([0.9401505 , 0.92864602, 0.93188967, 0.92251967, 0.9328494 ]), 171)
(array([0.94766238, 0.94649344, 0.96010202, 0.93743509, 0.94743315]), 128)
(array([0.92941194, 0.93696296, 0.95319408, 0.9221803 , 0.93095869]), 18)
(array([0.79241641, 0.79835925, 0.81870352, 0.81475369, 0.77936599]), 1)


In [73]:
for alpha in [0.008, 0.009, 0.01, 0.02, 0.03]:
    print(Lasso_select(X0_train,y0_train,alpha))

(array([0.93247107, 0.9395738 , 0.95584961, 0.92504159, 0.93555944]), 23)
(array([0.93111083, 0.93827466, 0.95457951, 0.92373417, 0.93331234]), 20)
(array([0.92941194, 0.93696296, 0.95319408, 0.9221803 , 0.93095869]), 18)
(array([0.91308236, 0.92398364, 0.93863707, 0.91083771, 0.90999924]), 6)
(array([0.90206943, 0.91264229, 0.92960934, 0.90314976, 0.89389083]), 4)


In [74]:
for alpha in [0.0095, 0.0096, 0.0097, 0.0098, 0.0099, 0.01]:
    print(Lasso_select(X0_train,y0_train,alpha))

(array([0.93032781, 0.93764108, 0.95384918, 0.92298488, 0.93211195]), 18)
(array([0.93014658, 0.93750867, 0.95371241, 0.92283178, 0.93187116]), 18)
(array([0.92996385, 0.93737434, 0.95358514, 0.92267327, 0.93164837]), 18)
(array([0.92978509, 0.93724124, 0.95345907, 0.92251251, 0.93142023]), 18)
(array([0.92959927, 0.93710306, 0.95332746, 0.92234948, 0.93119037]), 18)
(array([0.92941194, 0.93696296, 0.95319408, 0.9221803 , 0.93095869]), 18)


## Conclude 18 features is a good number for cluster 0.

In [75]:
for alpha in np.logspace(-5, -1, 5):
    print(Lasso_select(X1_train,y1_train,alpha))

(array([0.91450697, 0.92583915, 0.90804242, 0.92196817, 0.90714173]), 198)
(array([0.91663698, 0.93090324, 0.90961465, 0.92486307, 0.90933012]), 187)
(array([0.9220213 , 0.94236536, 0.91436338, 0.9370649 , 0.91458517]), 127)
(array([0.91885627, 0.91882881, 0.89608037, 0.93120482, 0.90748764]), 13)
(array([0.77716399, 0.77494101, 0.77344229, 0.78972065, 0.76185294]), 1)


In [76]:
for alpha in [0.008, 0.009, 0.01, 0.02, 0.03]:
    print(Lasso_select(X1_train,y1_train,alpha))

(array([0.9207557 , 0.92438973, 0.90010139, 0.93372401, 0.91037727]), 14)
(array([0.91988094, 0.92169996, 0.89814723, 0.93250621, 0.90889565]), 13)
(array([0.91885627, 0.91882881, 0.89608037, 0.93120482, 0.90748764]), 13)
(array([0.91003181, 0.9016044 , 0.88007828, 0.92248211, 0.89348594]), 4)
(array([0.90388038, 0.89406663, 0.8743715 , 0.91647788, 0.88592663]), 1)


In [78]:
for alpha in [0.0078, 0.0079, 0.008, 0.0081, 0.0082, 0.0083]:
    print(Lasso_select(X1_train,y1_train,alpha))

(array([0.92091136, 0.92487355, 0.90045318, 0.93395117, 0.91065072]), 16)
(array([0.92083469, 0.92463605, 0.90027662, 0.9338405 , 0.9105156 ]), 15)
(array([0.9207557 , 0.92438973, 0.90010139, 0.93372401, 0.91037727]), 14)
(array([0.92067603, 0.92414046, 0.89992317, 0.93360991, 0.91023573]), 14)
(array([0.92059382, 0.92388838, 0.89974187, 0.93349405, 0.91009099]), 14)
(array([0.92051029, 0.92362936, 0.89955375, 0.93337661, 0.90994211]), 14)


In [80]:
for alpha in [0.0073, 0.0074, 0.0075, 0.0076, 0.0077, 0.0078]:
    print(Lasso_select(X1_train,y1_train,alpha))

(array([0.92136314, 0.92606637, 0.90126985, 0.93460294, 0.91128054]), 18)
(array([0.92125095, 0.92580219, 0.90111264, 0.93443999, 0.91116003]), 18)
(array([0.9211432 , 0.92557029, 0.90095234, 0.9342978 , 0.91103763]), 17)
(array([0.92106101, 0.92534075, 0.90078892, 0.93417413, 0.91091138]), 17)
(array([0.92098669, 0.92510847, 0.9006226 , 0.93406372, 0.91078265]), 17)
(array([0.92091136, 0.92487355, 0.90045318, 0.93395117, 0.91065072]), 16)


## Conclude 18 features might be a good number for both clusters.

In [85]:
pipe = Pipeline(
        [
            ('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown = 'ignore'), categorical)], 
                                                remainder='passthrough')),
            ('scaler', StandardScaler(with_mean=False))
        ]
    )
    
X = pipe.fit_transform(X0_train)
y = y0_train

selector = SelectFromModel(Lasso(alpha=0.0095, max_iter=5000))
selector.fit(X,y)
mask = selector.get_support()
feat_names = pipe.named_steps['transformer'].get_feature_names()
names0 = [name for name, boo in zip(feat_names, mask) if boo]
names0

['Cat__x11_2Story',
 'Cat__x27_New',
 'OverallQual',
 'YearRemodAdd',
 'MasVnrArea',
 'ExterQual',
 'BsmtFinSF1',
 'HeatingQC',
 '1stFlrSF',
 '2ndFlrSF',
 'BsmtFullBath',
 'HalfBath',
 'TotRmsAbvGrd',
 'Fireplaces',
 'FireplaceQu',
 'GarageArea',
 'OpenPorchSF',
 'Comp']

In [86]:
pipe = Pipeline(
        [
            ('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown = 'ignore'), categorical)], 
                                                remainder='passthrough')),
            ('scaler', StandardScaler(with_mean=False))
        ]
    )
    
X = pipe.fit_transform(X1_train)
y = y1_train

selector = SelectFromModel(Lasso(alpha=0.0073, max_iter=5000))
selector.fit(X,y)
mask = selector.get_support()
feat_names = pipe.named_steps['transformer'].get_feature_names()
names1 = [name for name, boo in zip(feat_names, mask) if boo]
names1

['Cat__x11_1Story',
 'Cat__x14_BrkFace',
 'Cat__x18_None',
 'OverallCond',
 'YearRemodAdd',
 'BsmtQual',
 'BsmtExposure',
 'BsmtFinSF1',
 '1stFlrSF',
 'BsmtFullBath',
 'HalfBath',
 'TotRmsAbvGrd',
 'Fireplaces',
 'FireplaceQu',
 'GarageArea',
 'WoodDeckSF',
 'ScreenPorch',
 'Comp']

## Ridge for robustness.

In [87]:
dict(enumerate(categorical))

{0: 'MSZoning',
 1: 'Street',
 2: 'LotShape',
 3: 'LandContour',
 4: 'Utilities',
 5: 'LotConfig',
 6: 'LandSlope',
 7: 'Neighborhood',
 8: 'Condition1',
 9: 'Condition2',
 10: 'BldgType',
 11: 'HouseStyle',
 12: 'RoofStyle',
 13: 'RoofMatl',
 14: 'Exterior1st',
 15: 'Exterior2nd',
 16: 'MasVnrType',
 17: 'Foundation',
 18: 'BsmtFinType1',
 19: 'BsmtFinType2',
 20: 'Heating',
 21: 'CentralAir',
 22: 'Electrical',
 23: 'Functional',
 24: 'GarageType',
 25: 'Fence',
 26: 'MiscFeature',
 27: 'SaleType',
 28: 'SaleCondition'}

In [88]:
select0 = ['HouseStyle',
 'SaleType',
 'OverallQual',
 'YearRemodAdd',
 'MasVnrArea',
 'ExterQual',
 'BsmtFinSF1',
 'HeatingQC',
 '1stFlrSF',
 '2ndFlrSF',
 'BsmtFullBath',
 'HalfBath',
 'TotRmsAbvGrd',
 'Fireplaces',
 'FireplaceQu',
 'GarageArea',
 'OpenPorchSF',
 'Comp']

In [90]:
cats0 = [col for col in select0 if col in categorical]

In [92]:
X = X0_train[select0]
y = y0_train

pipe = Pipeline([('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown='ignore'), cats0)], 
                                            remainder='passthrough')),
                 ('scaler', StandardScaler()),
                 ('ridge', Ridge())])


param_grid = {'ridge__alpha':[0.001, 0.1, 1, 10]}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(pipe, param_grid, scoring='r2', cv=cv, n_jobs=-1)
grid.fit(X, y)

print(grid.cv_results_['mean_test_score'])
print(grid.best_params_)
print(grid.best_score_)

[0.9435184  0.94352135 0.94354421 0.94344473]
{'ridge__alpha': 1}
0.9435442106806213


In [97]:
X = X0_train[select0]
y = y0_train

pipe = Pipeline([('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown='ignore'), cats0)], 
                                            remainder='passthrough')),
                 ('scaler', StandardScaler()),
                 ('ridge', Ridge())])


param_grid = {'ridge__alpha':np.linspace(0.5, 5, 100)}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(pipe, param_grid, scoring='r2', cv=cv, n_jobs=-1)
grid.fit(X, y)

print(grid.cv_results_['mean_test_score'])
print(grid.best_params_)
print(grid.best_score_)

[0.94353239 0.94353356 0.9435347  0.94353583 0.94353694 0.94353803
 0.94353911 0.94354016 0.9435412  0.94354222 0.94354323 0.94354421
 0.94354518 0.94354613 0.94354706 0.94354797 0.94354887 0.94354975
 0.94355061 0.94355146 0.94355229 0.9435531  0.94355389 0.94355467
 0.94355542 0.94355617 0.94355689 0.9435576  0.94355829 0.94355896
 0.94355962 0.94356026 0.94356089 0.94356149 0.94356208 0.94356266
 0.94356321 0.94356375 0.94356428 0.94356478 0.94356528 0.94356575
 0.94356621 0.94356665 0.94356708 0.94356749 0.94356788 0.94356826
 0.94356862 0.94356897 0.94356929 0.94356961 0.94356991 0.94357019
 0.94357045 0.9435707  0.94357094 0.94357116 0.94357136 0.94357155
 0.94357172 0.94357188 0.94357202 0.94357215 0.94357226 0.94357235
 0.94357243 0.9435725  0.94357255 0.94357258 0.9435726  0.9435726
 0.94357259 0.94357257 0.94357252 0.94357247 0.9435724  0.94357231
 0.94357221 0.9435721  0.94357197 0.94357182 0.94357166 0.94357149
 0.9435713  0.9435711  0.94357088 0.94357065 0.9435704  0.94357

## Alpha of 3.73 looks good for cluster 0.

In [98]:
select1 = ['HouseStyle',
 'Exterior1st',
 'BsmtFinType1',
 'OverallCond',
 'YearRemodAdd',
 'BsmtQual',
 'BsmtExposure',
 'BsmtFinSF1',
 '1stFlrSF',
 'BsmtFullBath',
 'HalfBath',
 'TotRmsAbvGrd',
 'Fireplaces',
 'FireplaceQu',
 'GarageArea',
 'WoodDeckSF',
 'ScreenPorch',
 'Comp']

In [99]:
cats1 = [col for col in select1 if col in categorical]

In [100]:
X = X1_train[select1]
y = y1_train

pipe = Pipeline([('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown='ignore'), cats1)], 
                                            remainder='passthrough')),
                 ('scaler', StandardScaler()),
                 ('ridge', Ridge())])


param_grid = {'ridge__alpha':[0.001, 0.1, 1, 10]}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(pipe, param_grid, scoring='r2', cv=cv, n_jobs=-1)
grid.fit(X, y)

print(grid.cv_results_['mean_test_score'])
print(grid.best_params_)
print(grid.best_score_)

[0.92584317 0.92584624 0.92587134 0.9258704 ]
{'ridge__alpha': 1}
0.9258713394620706


In [102]:
X = X1_train[select1]
y = y1_train

pipe = Pipeline([('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown='ignore'), cats1)], 
                                            remainder='passthrough')),
                 ('scaler', StandardScaler()),
                 ('ridge', Ridge())])


param_grid = {'ridge__alpha':np.linspace(3, 7, 400)}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(pipe, param_grid, scoring='r2', cv=cv, n_jobs=-1)
grid.fit(X, y)

print(grid.cv_results_['mean_test_score'])
print(grid.best_params_)
print(grid.best_score_)

[0.92590939 0.92590952 0.92590966 0.92590979 0.92590991 0.92591004
 0.92591017 0.9259103  0.92591043 0.92591055 0.92591068 0.92591081
 0.92591093 0.92591105 0.92591118 0.9259113  0.92591142 0.92591155
 0.92591167 0.92591179 0.92591191 0.92591203 0.92591215 0.92591227
 0.92591238 0.9259125  0.92591262 0.92591273 0.92591285 0.92591296
 0.92591308 0.92591319 0.92591331 0.92591342 0.92591353 0.92591364
 0.92591375 0.92591386 0.92591397 0.92591408 0.92591419 0.9259143
 0.92591441 0.92591452 0.92591462 0.92591473 0.92591483 0.92591494
 0.92591504 0.92591515 0.92591525 0.92591535 0.92591545 0.92591555
 0.92591566 0.92591576 0.92591586 0.92591595 0.92591605 0.92591615
 0.92591625 0.92591634 0.92591644 0.92591654 0.92591663 0.92591673
 0.92591682 0.92591691 0.92591701 0.9259171  0.92591719 0.92591728
 0.92591737 0.92591746 0.92591755 0.92591764 0.92591773 0.92591782
 0.9259179  0.92591799 0.92591808 0.92591816 0.92591825 0.92591833
 0.92591841 0.9259185  0.92591858 0.92591866 0.92591874 0.92591

## Alpha of 5.37 looks good for cluster 1.

## Now putting it all together.

In [106]:
X0_tr = X0_train[select0]
X0_ts = X0_test[select0]

pipe = Pipeline([('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown='ignore'), cats0)], 
                                            remainder='passthrough')),
                 ('scaler', StandardScaler()),
                 ('ridge', Ridge(alpha=3.73))])

pipe.fit(X0_tr, y0_train)

cluster0_train_predict = pipe.predict(X0_tr)
cluster0_test_predict = pipe.predict(X0_ts)

X1_tr = X1_train[select1]
X1_ts = X1_test[select1]

pipe = Pipeline([('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown='ignore'), cats1)], 
                                            remainder='passthrough')),
                 ('scaler', StandardScaler()),
                 ('ridge', Ridge(alpha=5.37))])

pipe.fit(X1_tr, y1_train)

cluster1_train_predict = pipe.predict(X1_tr)
cluster1_test_predict = pipe.predict(X1_ts)

In [113]:
cluster0_train_predict = pd.DataFrame(cluster0_train_predict).rename(columns={0:'prediction'})
cluster1_train_predict = pd.DataFrame(cluster1_train_predict).rename(columns={0:'prediction'})
cluster0_test_predict = pd.DataFrame(cluster0_test_predict).rename(columns={0:'prediction'})
cluster1_test_predict = pd.DataFrame(cluster1_test_predict).rename(columns={0:'prediction'})

train_predict = pd.concat([cluster0_train_predict, cluster1_train_predict])
test_predict = pd.concat([cluster0_test_predict, cluster1_test_predict])
train_target = pd.concat([y0_train, y1_train])
test_target = pd.concat([y0_test, y1_test])

In [117]:
print(len(train_predict))
print(len(test_predict))
print(len(train_target))
print(len(test_target))

1871
624
1871
624


In [122]:
print(f'Train score is {r2_score(train_target, train_predict)}')
print(f'Test score is {r2_score(test_target, test_predict)}')

Train score is 0.9651636892887285
Test score is 0.8304349654373213


In [123]:
print(f'Cluster 0 train score is {r2_score(y0_train, cluster0_train_predict)}')
print(f'Cluster 0 test score is {r2_score(y0_test, cluster0_test_predict)}')
print('\n')
print(f'Cluster 1 train score is {r2_score(y1_train, cluster1_train_predict)}')
print(f'Cluster 1 test score is {r2_score(y1_test, cluster1_test_predict)}')

Cluster 0 train score is 0.9485168704918534
Cluster 0 test score is 0.8376971388950912


Cluster 1 train score is 0.9323360372676686
Cluster 1 test score is 0.5420071689880459
